In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.scenario.scenario import Tag
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.ConversionAnalyzer.AnalyzerErrorResult import AnalyzerErrorResult
from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.ConversionAnalyzer.SpotAnalyzer import SpotAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason
from OpenSCENARIO2CR.Osc2CrConverterResult import Osc2CrConverterResult
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.PlanningProblemBuilder import PlanningProblemSetBuilder

In [ ]:
scenario_path = ""
run_viewer = False
print_plots = False
plots_step = 5
plot_limit = 20  # If non-null the renderer follows the ego vehicle
following_obstacle_index = 0

In [ ]:
# Setup EsminiWrapper
esmini_wrapper = EsminiWrapperProvider(preferred_version="v2.26.5").provide_esmini_wrapper()
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False
esmini_wrapper.random_seed = 0

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = None
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)

In [ ]:
pps_builder = PlanningProblemSetBuilder()
pps_builder.goal_state_time_step = AbsRel(Interval(-10, 0), False)
pps_builder.goal_state_position_length = 50
pps_builder.goal_state_position_width = 10
pps_builder.goal_state_position_use_ego_rotation = True
pps_builder.goal_state_orientation = None
# pps_builder.goal_state_orientation = AbsRel(AngleInterval(-0.8, 0.8), False)
pps_builder.goal_state_velocity = None
# pps_builder.goal_state_velocity = AbsRel(Interval(-10, 10), False)

In [ ]:
converter = Osc2CrConverter(
    author="ADD AUTHOR HERE",
    affiliation="ADD AFFILIATION HERE",
    source="ADD SOURCE HERE",
    tags={Tag.SIMULATED},
)

converter.sim_wrapper = esmini_wrapper
converter.pps_builder = pps_builder

converter.dt_cr = 0.1
converter.keep_ego_vehicle = True
converter.trim_scenario = False
converter.use_implicit_odr_file = True
converter.analyzers = [
    EAnalyzer.SPOT_ANALYZER,
    EAnalyzer.DRIVABILITY_CHECKER,
    EAnalyzer.STL_MONITOR,
]

converter.dt_sim = 0.01
converter.odr_file_override = None
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE)

In [ ]:
result = converter.run_conversion(scenario_path)

In [ ]:
if isinstance(result, EFailureReason):
    print(f"Scenario failed due to {result}")
    assert False
result: Osc2CrConverterResult = result
for o_name,  analysis in result.analysis[EAnalyzer.SPOT_ANALYZER].items():
    if isinstance(analysis, AnalyzerErrorResult):
        print(f"{o_name}: {analysis.exception_text}\n{analysis.traceback_text}\n\n\n")